In [ ]:
def rle_encode(data):
    encoding = ''
    prev_char = ''
    count = 1
    if not data: return ''
    for char in data:
        if char != prev_char:
            if prev_char:
                encoding = encoding + str(count) + prev_char if count > 1 else encoding + prev_char
            count = 1
            prev_char = char
        else:
            count += 1
    else:
        encoding = encoding + str(count) + prev_char if count > 1 else encoding + prev_char
        return encoding

def m_to_rle(m):
    
    prefix = 'x = 0, y = 0, rule = B3/S23 ' 
    m = [''.join(list(map(str, i)) + ['$',]) for i in m]
    S = ''.join(m)
    S = S.replace('0', 'b')
    S = S.replace('1', 'o')
    return rle_encode(prefix + S + '!')

def mat2_to_rle(m):
    new_m = [[0]*len(m) for _ in range(len(m))]
    a,b = 0,0

    for i in range(len(m)):
        for j in range(len(m)):
            if m[i][j] == 2:
                new_m[i][j] = 0
                a = i
                b = j
            else:
                new_m[i][j] = m[i][j]

    return f'{a},{b} ' + m_to_rle(new_m)

def next_state(alive_n, ambiguous_n, center):
    if center == 2: 
        if alive_n == 3 and ambiguous_n == 0:
            return 1
        if alive_n > 3:
            return 0
        if alive_n + ambiguous_n < 2:
            return 0
        return 2

    neighbours = set(range(alive_n, alive_n + ambiguous_n + 1))
    if center == 0:
        target = {3}
    else:
        target = {2,3}

    if neighbours.isdisjoint(target):
        return 0
    if neighbours <= target:
        return 1
        
    return 2

In [ ]:
def has_3_on_boarder_square(arr):
    tr = len(arr)
    for j in range(2, tr):
        if arr[j-2][ 0] == 1 and arr[j-1][0] == 1 and arr[j][ 0] == 1:
            return True
        if arr[j-2][ tr-1] == 1 and arr[j-1][ tr-1] == 1 and arr[j][ tr-1] == 1:
            return True
        if arr[0][ j-2] == 1 and arr[0][ j-1] == 1 and arr[0][ j] == 1:
            return True
        if arr[tr-1][ j-2] == 1 and arr[tr-1][ j-1] == 1 and arr[tr-1][ j] == 1:
            return True
    return False

In [ ]:
from os import readlink
from copy import deepcopy
from random import shuffle, sample, randint

import numpy as np

PERIOD = 2
TARGET_DIM = 5
TARGET_POP = TARGET_DIM**2

def near(a, b):
    yield (a, b+1)
    yield (a+1, b)
    yield (a, b-1)
    yield (a-1, b)
    yield (a+1, b+1)
    yield (a-1, b+1)
    yield (a+1, b-1)
    yield (a-1, b-1)

def generate_rotrefl(tile):
    tile = np.array(tile)
    out = [tile.tolist(),]
    
    #if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    
    tile = np.rot90(tile)

    if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    

    tile = np.rot90(tile)

    if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    

    tile = np.rot90(tile)

    if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    
    return out



def get_map():
    d = dict()
    for i in range(9):
        for j in range(9):
            if i+j>8:continue
            for c in range(3):
                d[(i,j,c)] = next_state(i,j,c)
    return d

transitions = get_map()
#print(transitions)

def get_new_gen(m):
    n_of_certain = 0
    tr = len(m)
    new_m = [[0]*tr for i in range(tr)]

    for i in range(tr):
        for j in range(tr):
            n_1 = 0
            n_2 = 0
            for a,b in near(i,j):
                if 0 <= a < tr and 0 <= b <tr:
                    if m[a][b] == 2:
                        n_2 += 1
                    elif m[a][b] == 1 :
                        n_1 += 1
            new_state = transitions[(n_1, n_2, m[i][j] )]
            if new_state!=2:n_of_certain+=1
            new_m[i][j] = new_state

    has_3 = has_3_on_boarder_square(new_m)

    return new_m, n_of_certain, has_3

def analize_mat(m):
    counts = []
    un_amb = 0
    tr = len(m)
    old_m = deepcopy(m)

    for i in range(PERIOD):
        m, n_of_certain, has_3_on_b = get_new_gen(m)

        if has_3_on_b:
            return False
        if n_of_certain == 0:
            return (sum(counts[:]), 0)

        counts.append(n_of_certain)

    for i in range(tr):
        for j in range(tr):
            if m[i][j] != 2:
                if m[i][j] == old_m[i][j]:
                    un_amb += 1
                else:
                    return False
    
    if un_amb == TARGET_POP:
        m, n_of_certain, has_3_on_b = get_new_gen(deepcopy(old_m))
        if old_m == m:
            return False

    return (sum(counts[:-1]), un_amb)

def hash_t(s):
    #t = sorted(generate_rotrefl(s))[0]
    return ''.join(''.join(map(str, i)) for i in s)

out = [[], ]
max_l = [0,]
seen = set()
ret = [0,]
too_many_iters = [0,]


def dfs(mat, free_moves, n = 0):
    # if n > 2:
    #     h = hash_t(mat)
    #     seen.add(h)

    too_many_iters[0]  =  too_many_iters[0] + 1

    if n >= max_l[-1]:
        print(n, len(seen), mat2_to_rle(mat))
        max_l[0] = n
        out[0] = mat

    if ret[0] == 1:
        return
   
    possible_m = []

    summ_p = 0

    for i in range(TARGET_POP):
        if not free_moves[i]: continue

        a = i % TARGET_DIM
        b = i //TARGET_DIM

        new_mat = deepcopy(mat)
        new_mat[a][b] = 1

        h = hash_t(new_mat)
        if h not in seen:
            #seen.add(h)
            eval_qual = analize_mat(new_mat)
            if not(eval_qual is False): 
                p = 1 + eval_qual[0] + 1000*eval_qual[1]
                summ_p += p
                possible_m.append((p, i, new_mat))

        new_mat2 = deepcopy(mat)
        new_mat2[a][b] = 0

        h = hash_t(new_mat2)
        if h not in seen:
            
            #seen.add(h)
            eval_qual = analize_mat(new_mat2)
            if not(eval_qual is False): 
                p =  1  + eval_qual[0] + 1000*eval_qual[1]
                summ_p += p
                possible_m.append((p, i, new_mat2))
    
    #N_branches = max(20, min(5, sum([1 for i in possible_m if i[0]>=1000])))
    #N_branches = randint(3, 9)
    N_branches = 1

    #possible_m = sorted(possible_m, reverse=True)
    #shuffle(possible_m)

    if summ_p > 0:
        #print([t[0]/summ_p for t in possible_m])
        n_of_choosen = np.random.choice(len(possible_m), min(N_branches, len(possible_m)), p=[t[0]/summ_p for t in possible_m], replace = False)
        # if n == 0 or n == 2:
        #     print(n, n_of_choosen)
    else:
        n_of_choosen = sample( range(len(possible_m)), min(N_branches, len(possible_m)) )

    new_braches = [possible_m[t] for t in n_of_choosen]

    #new_braches = possible_m

    for k in new_braches:
        item = k
        new_moves = free_moves.copy()
        new_moves[item[1]] = False
        dfs(item[2], new_moves, n+1)
    
    # if n == TARGET_POP-1:
    #     ret[0] = 1

from tqdm import tqdm

for i in tqdm(range(1000000)):
    
    ret = [0,]
    too_many_iters= [0,]
    start_m = [[2]*TARGET_DIM for i in range(TARGET_DIM)]
    start_moves = [True]*TARGET_POP

    dfs(start_m, start_moves, 0)



In [ ]:
from copy import deepcopy
from random import shuffle, sample, randint, random

import numpy as np

PERIOD = 19
TARGET_DIM = 17
TARGET_POP = TARGET_DIM**2

def near(a, b):
    yield (a, b+1)
    yield (a+1, b)
    yield (a, b-1)
    yield (a-1, b)
    yield (a+1, b+1)
    yield (a-1, b+1)
    yield (a+1, b-1)
    yield (a-1, b-1)

def generate_rotrefl(tile):
    tile = np.array(tile)
    out = [tile.tolist(),]
    
    #if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    
    tile = np.rot90(tile)

    if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    

    tile = np.rot90(tile)

    if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    

    tile = np.rot90(tile)

    if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    
    return out



def get_map():
    d = dict()
    for i in range(9):
        for j in range(9):
            if i+j>8:continue
            for c in range(3):
                d[(i,j,c)] = next_state(i,j,c)
    return d

transitions = get_map()
#print(transitions)

def get_new_gen(m):
    n_of_certain = 0
    tr = len(m)
    new_m = [[0]*tr for i in range(tr)]

    for i in range(tr):
        for j in range(tr):
            n_1 = 0
            n_2 = 0
            for a,b in near(i,j):
                if 0 <= a < tr and 0 <= b <tr:
                    if m[a][b] == 2:
                        n_2 += 1
                    elif m[a][b] == 1 :
                        n_1 += 1
            new_state = transitions[(n_1, n_2, m[i][j] )]
            if new_state!=2:n_of_certain+=1
            new_m[i][j] = new_state

    has_3 = has_3_on_boarder_square(new_m)

    return new_m, n_of_certain, has_3

def analize_mat(m):
    counts = []
    un_amb = 0
    tr = len(m)
    old_m = deepcopy(m)

    for i in range(PERIOD):
        m, n_of_certain, has_3_on_b = get_new_gen(m)

        if has_3_on_b:
            return False
        if n_of_certain == 0:
            return (sum(counts[:]), 0)

        counts.append(n_of_certain)

    for i in range(tr):
        for j in range(tr):
            if m[i][j] != 2:
                if m[i][j] == old_m[i][j]:
                    un_amb += 1
                else:
                    return False
    
    if un_amb == TARGET_POP:
        m, n_of_certain, has_3_on_b = get_new_gen(deepcopy(old_m))
        if old_m == m:
            return False
        else:
            print('yay! \n')
  
    return (sum(counts[:-1]), un_amb)

def hash_t(s):
    t = sorted(generate_rotrefl(s))[0]
    return ''.join(''.join(map(str, i)) for i in t)

out = [[], ]
max_l = [0,]
seen = set()

starting_points = []


def dfs(mat, n = 0):
    free_moves = []
   

    for i in range(len(mat)):
        for j in range(len(mat)):
            if mat[i][j] == 2:
                free_moves.append((i,j))

    if n > max_l[-1] or n == TARGET_POP:
        print(n, len(seen), len(starting_points), mat2_to_rle(mat))
        max_l[0] = n
        out[0] = mat

    possible_m = []
    summ_p = 0

    for a, b in free_moves:
        new_mat = deepcopy(mat)
        new_mat[a][b] = 1

        h = hash_t(new_mat)
        if h not in seen:
            seen.add(h)
            eval_qual = analize_mat(new_mat)
            if not(eval_qual is False): 
                p = 1 + (10000*eval_qual[1]/(eval_qual[0]+1))
                summ_p += p
                possible_m.append((n, p, new_mat))

        new_mat2 = deepcopy(mat)
        new_mat2[a][b] = 0

        h = hash_t(new_mat2)
        if h not in seen:
            seen.add(h)
            eval_qual = analize_mat(new_mat2)
            if not(eval_qual is False): 
                p =  1 + (10000*eval_qual[1]/(eval_qual[0]+1))
                summ_p += p
                possible_m.append((n, p, new_mat2))


    

    # #shuffle(possible_m)
    # if summ_p > 0:
    #     #print([t[0]/summ_p for t in possible_m])
    #     n_of_choosen = np.random.choice(len(possible_m), min(N_branches, len(possible_m)), p=[t[0]/summ_p for t in possible_m], replace = False)
    #     # if n == 0 or n == 2:
    #     #     print(n, n_of_choosen)
    # else:
    #     n_of_choosen = sample( range(len(possible_m)), min(N_branches, len(possible_m)) )
    # new_braches = [possible_m[t] for t in n_of_choosen]

    if len(possible_m) > 0:
        n_of_chosen = np.random.choice(len(possible_m),1, p=[t[1]/summ_p for t in possible_m])[0]
        br_t = possible_m.pop(n_of_chosen)

        if len(possible_m) > 1:
            starting_points.extend(possible_m)

        
        dfs(br_t[2], n+1, )

from tqdm import tqdm

start_m = [[2]*TARGET_DIM for i in range(TARGET_DIM)]
dfs(start_m, 0)

for i in tqdm(range(1000000)):
    if len(starting_points) == 0:break
    starting_points = sorted(starting_points)

    # if len(starting_points) < 100:
    #     start_pack = starting_points.pop(0)
    # else:
    #     start_pack = starting_points.pop(-1)

    if random()<0.2:
        summ_p1 = sum(t[1] for t in starting_points)
        start_p1 = np.random.choice(len(starting_points), 1, p=[t[1]/summ_p1 for t in starting_points])
        start_pack = starting_points.pop(start_p1[0])
    else:
        start_pack = starting_points.pop(randint(0, min(100, len(starting_points)-1)))
    

    dfs(start_pack[2], start_pack[0]+1)



1 90 89 16,16 x = 0, y = 0, rule = B3/S23 17b$17b$17b$6bo10b$17b$17b$17b$17b$17b$17b$17b$17b$17b$17b$17b$17b$17b$!
2 665 663 16,16 x = 0, y = 0, rule = B3/S23 17b$17b$17b$6bo10b$17b$17b$17b$17b$5bo11b$17b$17b$17b$17b$17b$17b$17b$17b$!
3 1239 1236 16,16 x = 0, y = 0, rule = B3/S23 17b$17b$17b$6bo10b$17b$17b$17b$17b$5bo11b$17b$17b$17b$17b$17b$17b$17b$17b$!
4 1811 1807 16,16 x = 0, y = 0, rule = B3/S23 17b$17b$17b$6bo10b$17b$17b$17b$17b$5bo11b$17b$17b$17b$17b$17b$9bo7b$17b$17b$!
5 2381 2376 16,16 x = 0, y = 0, rule = B3/S23 17b$17b$17b$6bo10b$17b$17b$17b$17b$5bo11b$17b$17b$17b$17b$17b$9bo7b$17b$17b$!
6 2949 2943 16,16 x = 0, y = 0, rule = B3/S23 17b$17b$17b$6bo10b$17b$17b$17b$17b$5bo11b$17b$17b$17b$17b$17b$9bo7b$17b$17b$!
7 3515 3508 16,16 x = 0, y = 0, rule = B3/S23 17b$16bo$17b$6bo10b$17b$17b$17b$17b$5bo11b$17b$17b$17b$17b$17b$9bo7b$17b$17b$!
8 4079 4071 16,16 x = 0, y = 0, rule = B3/S23 17b$16bo$17b$6bo10b$17b$17b$17b$17b$5bo11b$17b$bo15b$17b$17b$17b$9bo7b$17b$17b$!
9 4641 4632 16,16 x

  0%|          | 3/1000000 [12:24<66924:38:04, 240.93s/it]

286 291940 287601 15,16 x = 0, y = 0, rule = B3/S23 ob2obobobo2bobo2b$5o2bo3bob2obo$3b2obob3o2b2obo$4b5obo2bo2bo$b2ob2ob2o6bob$o6b2obobo2b2o$b2obob3o2b3o2bo$b3obobob5ob2o$4b2obobo2b3obo$3b3o3b2obo4b$3o2b2ob2o2bo2bob$2bob2ob2o4b3ob$2obobob3o2b3o2b$bobo2bob3o3b2ob$2ob2o3b4o4bo$3ob2obob2ob2obob$bob6obo4bob$!


  0%|          | 6/1000000 [25:05<69356:35:53, 249.69s/it]

287 541002 533937 14,10 x = 0, y = 0, rule = B3/S23 4bo2bobobo3bob$o2bobo7b4o$3o3b2o2bob3o2b$3bo3bo3bobo3b$2bobobo2bob2o4b$bob2obobo3b3o2b$5bo2b7obo$2bob2ob2o3b2o2bo$2b2obo4bobob3o$o3b2o3b3o3b2o$8ob6o2b$b3obo3b2obob3o$2o2b4o2bo3bobo$obob4obob2o2b2o$ob3ob2obob2ob2ob$bobo5b5o2bo$2obobob2o2bobobob$!


  0%|          | 22/1000000 [1:24:43<64185:10:53, 231.07s/it]


KeyboardInterrupt: ignored

In [ ]:
from copy import deepcopy
from random import shuffle, sample, randint, random

import numpy as np

PERIOD = 3
TARGET_DIM = 7
TARGET_POP = TARGET_DIM**2

def near(a, b):
    yield (a, b+1)
    yield (a+1, b)
    yield (a, b-1)
    yield (a-1, b)
    yield (a+1, b+1)
    yield (a-1, b+1)
    yield (a+1, b-1)
    yield (a-1, b-1)

def generate_rotrefl(tile):
    tile = np.array(tile)
    out = [tile.tolist(),]
    
    #if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    
    tile = np.rot90(tile)

    if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    

    tile = np.rot90(tile)

    if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    

    tile = np.rot90(tile)

    if tile.tolist() not in out:out.append(tile.tolist())
    if np.fliplr(tile).tolist() not in out:out.append(np.fliplr(tile).tolist())
    
    return out



def get_map():
    d = dict()
    for i in range(9):
        for j in range(9):
            if i+j>8:continue
            for c in range(3):
                d[(i,j,c)] = next_state(i,j,c)
    return d

transitions = get_map()
#print(transitions)

def get_new_gen(m):
    n_of_certain = 0
    tr = len(m)
    new_m = [[0]*tr for i in range(tr)]

    for i in range(tr):
        for j in range(tr):
            n_1 = 0
            n_2 = 0
            for a,b in near(i,j):
                if 0 <= a < tr and 0 <= b <tr:
                    if m[a][b] == 2:
                        n_2 += 1
                    elif m[a][b] == 1 :
                        n_1 += 1
            new_state = transitions[(n_1, n_2, m[i][j] )]
            if new_state!=2:n_of_certain+=1
            new_m[i][j] = new_state

    has_3 = has_3_on_boarder_square(new_m)

    return new_m, n_of_certain, has_3

def analize_mat(m):
    counts = []
    un_amb = 0
    tr = len(m)
    old_m = deepcopy(m)

    for i in range(PERIOD):
        m, n_of_certain, has_3_on_b = get_new_gen(m)

        if has_3_on_b:
            return False
        if n_of_certain == 0:
            return (sum(counts[:]), 0)

        counts.append(n_of_certain)

    for i in range(tr):
        for j in range(tr):
            if m[i][j] != 2:
                if m[i][j] == old_m[i][j]:
                    un_amb += 1
                else:
                    return False
    
    if un_amb == TARGET_POP:
        m, n_of_certain, has_3_on_b = get_new_gen(deepcopy(old_m))
        if old_m == m:
            return False
        else:
            print('yay! \n', old_m)
  
    return (sum(counts[:-1]), un_amb)

def hash_t(s):
    t = sorted(generate_rotrefl(s))[0]
    return ''.join(''.join(map(str, i)) for i in t)

out = [[], ]
max_l = [0,]
seen = set()

starting_points = []

def dfs(mat, free_moves, n = 0, p= 0):

    if n > max_l[-1] or n == TARGET_POP:
        print(n, p, mat2_to_rle(mat))
        max_l[0] = n
        out[0] = mat

    possible_m = []
    summ_p = 0
    #print('aaaa', free_moves)

    for a, b in free_moves:
        new_mat = deepcopy(mat)
        new_mat[a][b] = 1
        eval_qual = analize_mat(new_mat)
        if not(eval_qual is False): 
            p = 1 + (10000*eval_qual[1]/(eval_qual[0]+1))
            #p = 1
            summ_p += p
            possible_m.append((p, new_mat,a,b))

        new_mat2 = deepcopy(mat)
        new_mat2[a][b] = 0
        eval_qual = analize_mat(new_mat2)
        if not(eval_qual is False): 
            p =  1  +  (10000*eval_qual[1]/(eval_qual[0]+1))
            #p = 1
            summ_p += p
            possible_m.append((p, new_mat2, a,b))

    if len(possible_m) > 0:
        #print(summ_p, sum([t[0]/summ_p for t in possible_m]))
        n_of_chosen = np.random.choice(len(possible_m), 1, p=[t[0]/summ_p for t in possible_m])[0]
        br_t = possible_m[n_of_chosen]
        free_moves.remove((br_t[2], br_t[3]))
        dfs(br_t[1], free_moves, n+1, br_t[0])

from tqdm import tqdm

for i in tqdm(range(1000000)):
    start_m = [[2]*TARGET_DIM for i in range(TARGET_DIM)]
    start_moves = [(a,b) for a in range(TARGET_DIM) for b in range(TARGET_DIM)]
    #print(start_moves)
    dfs(start_m, start_moves)
    




  0%|          | 0/1000000 [00:00<?, ?it/s]

1 1.0 6,6 x = 0, y = 0, rule = B3/S23 7b$7b$7b$7b$3bo3b$7b$7b$!
2 1.0 6,6 x = 0, y = 0, rule = B3/S23 7b$7b$7b$7b$3bobob$7b$7b$!
3 1.0 6,6 x = 0, y = 0, rule = B3/S23 7b$7b$7b$7b$3bobob$7b$7b$!
4 1.0 6,6 x = 0, y = 0, rule = B3/S23 7b$7b$7b$7b$3bobob$7b$4bo2b$!
5 1.0 6,6 x = 0, y = 0, rule = B3/S23 7b$7b$7b$7b$3bobob$7b$4bo2b$!
6 1.0 6,6 x = 0, y = 0, rule = B3/S23 7b$7b$7b$5bob$3bobob$7b$4bo2b$!
7 1.0 6,6 x = 0, y = 0, rule = B3/S23 7b$7b$7b$5bob$3bobob$7b$4bo2b$!
8 1.0 6,6 x = 0, y = 0, rule = B3/S23 7b$7b$7b$5bob$3bobob$7b$4bo2b$!
9 1.0 6,6 x = 0, y = 0, rule = B3/S23 7b$7b$7b$5bob$o2bobob$7b$4bo2b$!
10 1.0 6,6 x = 0, y = 0, rule = B3/S23 7b$7b$7b$5bob$o2bobob$7b$4bo2b$!
11 1.0 6,5 x = 0, y = 0, rule = B3/S23 7b$7b$7b$5bob$o2bobob$7b$4bobo$!
12 1.0 6,5 x = 0, y = 0, rule = B3/S23 7b$7b$7b$5b2o$o2bobob$7b$4bobo$!
13 1.0 6,5 x = 0, y = 0, rule = B3/S23 7b$7b$7b$5b2o$o2bobob$7b$4bobo$!
14 1.0 6,5 x = 0, y = 0, rule = B3/S23 7b$7b$7b$o4b2o$o2bobob$7b$4bobo$!
15 1.0 6,5 x = 0, y = 0, rul

  0%|          | 1/1000000 [00:01<338:45:49,  1.22s/it]

33 1.0 6,3 x = 0, y = 0, rule = B3/S23 bo3bob$2b2o3b$b2o2b2o$o4b2o$o2bobob$bo4bo$bo2bobo$!
34 953.3809523809524 6,3 x = 0, y = 0, rule = B3/S23 bo3bob$2b2o3b$b2o2b2o$o4b2o$o2bobob$bo4bo$bo2bobo$!
35 953.3809523809524 6,3 x = 0, y = 0, rule = B3/S23 bo3bob$2b2o3b$b2o2b2o$o4b2o$o2bobob$bo4bo$bo2bobo$!
36 953.3809523809524 6,2 x = 0, y = 0, rule = B3/S23 bo3bob$2b2o3b$b2o2b2o$o4b2o$o2bobob$bo4bo$bo2bobo$!
37 953.3809523809524 6,0 x = 0, y = 0, rule = B3/S23 bo3bob$2b2o3b$b2o2b2o$o4b2o$o2bobob$bo4bo$bo2bobo$!
38 870.5652173913044 6,0 x = 0, y = 0, rule = B3/S23 bo2b2ob$2b2o3b$b2o2b2o$o4b2o$o2bobob$bo4bo$bo2bobo$!
39 834.3333333333334 6,0 x = 0, y = 0, rule = B3/S23 bo2b2ob$2b2o3b$b2o2b2o$o4b2o$o2bobob$bo4bo$bo2bobo$!
40 770.2307692307693 6,0 x = 0, y = 0, rule = B3/S23 bo2b2ob$2b2o3b$b2o2b2o$o4b2o$o2bobob$bo4bo$bo2bobo$!
41 715.2857142857143 6,0 x = 0, y = 0, rule = B3/S23 bo2b2ob$2b2o3b$b2o2b2o$o4b2o$o2bob2o$bo4bo$bo2bobo$!
42 626.0 5,5 x = 0, y = 0, rule = B3/S23 bo2b2ob$2b2o3b$b2o2b2o$o

  0%|          | 3/1000000 [00:03<291:14:21,  1.05s/it]

46 1112.111111111111 4,5 x = 0, y = 0, rule = B3/S23 3bobob$3bo2bo$4bo2b$2b4ob$ob2o3b$3bob2o$o3bobo$!


  0%|          | 26/1000000 [00:26<284:06:47,  1.02s/it]

47 1482.4814814814815 5,4 x = 0, y = 0, rule = B3/S23 3b2obo$5bob$bobo2bo$2b3o2b$b2o2bob$5b2o$2bobo2b$!


  0%|          | 80/1000000 [01:20<278:49:42,  1.00s/it]


KeyboardInterrupt: ignored